# Forward Feature Selection

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
path = r"https://raw.githubusercontent.com/sindhura-nk/Datasets/refs/heads/main/Cars93.csv"
import pandas as pd
df = pd.read_csv(path,na_values=['','NA'],keep_default_na=False)
df.head()

,id,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


# perform basic data quality checks

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  93 non-null     int64  
 1   Manufacturer        93 non-null     object 
 2   Model               93 non-null     object 
 3   Type                93 non-null     object 
 4   Min.Price           93 non-null     float64
 5   Price               93 non-null     float64
 6   Max.Price           93 non-null     float64
 7   MPG.city            93 non-null     int64  
 8   MPG.highway         93 non-null     int64  
 9   AirBags             89 non-null     object 
 10  DriveTrain          93 non-null     object 
 11  Cylinders           93 non-null     object 
 12  EngineSize          93 non-null     float64
 13  Horsepower          93 non-null     int64  
 14  RPM                 93 non-null     int64  
 15  Rev.per.mile        93 non-null     int64  
 16  Man.trans.

In [6]:
df.shape 

(93, 28)

#  Separate X and Y features
    Y: Weight
    X: apart from Weight and ID, considering all the features

In [7]:
df.columns

Index(['id', 'Manufacturer', 'Model', 'Type', 'Min.Price', 'Price',
       'Max.Price', 'MPG.city', 'MPG.highway', 'AirBags', 'DriveTrain',
       'Cylinders', 'EngineSize', 'Horsepower', 'RPM', 'Rev.per.mile',
       'Man.trans.avail', 'Fuel.tank.capacity', 'Passengers', 'Length',
       'Wheelbase', 'Width', 'Turn.circle', 'Rear.seat.room', 'Luggage.room',
       'Weight', 'Origin', 'Make'],
      dtype='object')

In [31]:
X = df.drop(columns=['id','Weight'])
Y = df[['Weight']]

In [32]:
X.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,Front,...,13.2,5,177,102,68,37,26.5,11.0,non-USA,Acura Integra
1,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,...,18.0,5,195,115,71,38,30.0,15.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,Front,...,16.9,5,180,102,67,37,28.0,14.0,non-USA,Audi 90
3,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,Front,...,21.1,6,193,106,70,37,31.0,17.0,non-USA,Audi 100
4,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,...,21.1,4,186,109,69,39,27.0,13.0,non-USA,BMW 535i


In [33]:
Y.head()

,Weight
0,2705
1,3560
2,3375
3,3405
4,3640


In [34]:
X.isna().sum()

Manufacturer           0
Model                  0
Type                   0
Min.Price              0
Price                  0
Max.Price              0
MPG.city               0
MPG.highway            0
AirBags                4
DriveTrain             0
Cylinders              0
EngineSize             0
Horsepower             0
RPM                    0
Rev.per.mile           0
Man.trans.avail        0
Fuel.tank.capacity     0
Passengers             0
Length                 0
Wheelbase              0
Width                  0
Turn.circle            0
Rear.seat.room         2
Luggage.room          11
Origin                 0
Make                   0
dtype: int64

# Data Cleaning and Data preprocessing

In [43]:
cat = list(X.columns[X.dtypes=='object'])
con = list(X.columns[X.dtypes!='object'])

In [44]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

In [45]:
num_pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    OrdinalEncoder()
)

In [46]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OrdinalEncoder()

)

In [47]:
pre = ColumnTransformer([
    ("cat",cat_pipe,cat),
    ("con",num_pipe,con)
]).set_output(transform="pandas")

In [48]:
pre

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder())]),
                                 ['Manufacturer', 'Model', 'Type', 'AirBags',
                                  'DriveTrain', 'Cylinders', 'Man.trans.avail',
                                  'Origin', 'Make']),
                                ('con',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder())]),
                                 ['Min.Price', 'Price', 'Max.Price', 'MPG.city',
                                  'MPG.highway', 'EngineSize', 'Horsepower',
                                  'RPM', 'Rev.per.mile', 'Fuel.tank.capacity',
                                  'Passengers', 'Length', 'Wheelbase', 'Width',
                                  'Turn.circle', 'Rear.seat.room',
                                  'Luggage.room'])])

In [49]:
X_pre = pre.fit_transform(X)
X_pre.head()

,cat__Manufacturer,cat__Model,cat__Type,cat__AirBags,cat__DriveTrain,cat__Cylinders,cat__Man.trans.avail,cat__Origin,cat__Make,con__Min.Price,...,con__RPM,con__Rev.per.mile,con__Fuel.tank.capacity,con__Passengers,con__Length,con__Wheelbase,con__Width,con__Turn.circle,con__Rear.seat.room,con__Luggage.room
0,0.0,48.0,3.0,2.0,1.0,1.0,1.0,1.0,0.0,25.0,...,22.0,67.0,7.0,2.0,18.0,10.0,6.0,5.0,9.0,4.0
1,0.0,55.0,2.0,0.0,1.0,3.0,1.0,1.0,1.0,67.0,...,13.0,37.0,23.0,2.0,36.0,23.0,9.0,6.0,17.0,9.0
2,1.0,8.0,0.0,1.0,1.0,3.0,1.0,1.0,3.0,63.0,...,13.0,32.0,20.0,2.0,21.0,10.0,5.0,5.0,13.0,8.0
3,1.0,0.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,69.0,...,13.0,52.0,34.0,3.0,34.0,14.0,8.0,5.0,19.0,11.0
4,2.0,5.0,2.0,1.0,2.0,1.0,1.0,1.0,4.0,61.0,...,16.0,54.0,34.0,1.0,27.0,17.0,7.0,7.0,10.0,6.0


# Feature Selection: Forward Selection

In [51]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

In [54]:
base_model = LinearRegression()
for_sel = SequentialFeatureSelector(base_model,direction='forward',n_features_to_select=10)
for_sel.fit(X_pre, Y)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=10)

In [55]:
for_sel.get_feature_names_out()

array(['cat__Cylinders', 'cat__Origin', 'con__MPG.highway',
       'con__Horsepower', 'con__Rev.per.mile', 'con__Fuel.tank.capacity',
       'con__Length', 'con__Wheelbase', 'con__Width',
       'con__Rear.seat.room'], dtype=object)

In [57]:
imp_cols = for_sel.get_feature_names_out()
imp_cols

array(['cat__Cylinders', 'cat__Origin', 'con__MPG.highway',
       'con__Horsepower', 'con__Rev.per.mile', 'con__Fuel.tank.capacity',
       'con__Length', 'con__Wheelbase', 'con__Width',
       'con__Rear.seat.room'], dtype=object)

In [58]:
len(imp_cols)

10

In [59]:
sel_cols = []
for i in imp_cols:
    cols = i.split('__')[-1]
    sel_cols.append(cols)
sel_cols

['Cylinders',
 'Origin',
 'MPG.highway',
 'Horsepower',
 'Rev.per.mile',
 'Fuel.tank.capacity',
 'Length',
 'Wheelbase',
 'Width',
 'Rear.seat.room']

In [61]:
X_sel = X[sel_cols]
X_sel.head()

,Cylinders,Origin,MPG.highway,Horsepower,Rev.per.mile,Fuel.tank.capacity,Length,Wheelbase,Width,Rear.seat.room
0,4,non-USA,31,140,2890,13.2,177,102,68,26.5
1,6,non-USA,25,200,2335,18.0,195,115,71,30.0
2,6,non-USA,26,172,2280,16.9,180,102,67,28.0
3,6,non-USA,26,172,2535,21.1,193,106,70,31.0
4,4,non-USA,30,208,2545,21.1,186,109,69,27.0


# 2nd step Data preprocessing

In [62]:
from sklearn.preprocessing import OneHotEncoder


In [64]:
X_sel_cat = list(X_sel.columns[X_sel.dtypes=='object'])
X_sel_con = list(X_sel.columns[X_sel.dtypes!='object'])

In [65]:
num_pipe2 = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

In [67]:
cat_pipe2 = make_pipeline(
    SimpleImputer(strategy= 'most_frequent'),
    OneHotEncoder(handle_unknown='ignore',sparse_output=False)
)

In [69]:
pre2 = ColumnTransformer([
    ('cat',cat_pipe2,X_sel_cat),
    ('con',num_pipe2,X_sel_con)
]).set_output(transform='pandas')

In [70]:
pre2

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Cylinders', 'Origin']),
                                ('con',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['MPG.highway', 'Horsepower', 'Rev.per.mile',
                                  'Fuel.tank.capacity', 'Length', 'Wheelbase',
                                  'Width', 'Rear.seat.room'])])

In [71]:
X_sel_pre = pre2.fit_transform(X_sel)
X_sel_pre.head()

,cat__Cylinders_3,cat__Cylinders_4,cat__Cylinders_5,cat__Cylinders_6,cat__Cylinders_8,cat__Cylinders_rotary,cat__Origin_USA,cat__Origin_non-USA,con__MPG.highway,con__Horsepower,con__Rev.per.mile,con__Fuel.tank.capacity,con__Length,con__Wheelbase,con__Width,con__Rear.seat.room
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.360925,-0.073484,1.129530,-1.062184,-0.427186,-0.286932,-0.366184,-0.452197
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.770514,1.078322,0.005661,0.409445,0.812171,1.629649,0.431983,0.738090
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.581941,0.540813,-0.105713,0.072197,-0.220626,-0.286932,-0.632239,0.057926
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.581941,0.540813,0.410659,1.359872,0.674465,0.302785,0.165927,1.078172
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.172352,1.231897,0.430909,1.359872,0.192493,0.745073,-0.100128,-0.282156


# Final model building

In [74]:
model= LinearRegression()
model.fit(X_sel_pre,Y)

LinearRegression()

In [76]:
model.intercept_

array([3057.59799091])

In [77]:
model.coef_

array([[ 213.46853302,    2.6856377 ,  146.43642006,   40.01867597,
         -59.87825594, -342.73101082,  -40.06266479,   40.06266479,
        -145.04401857,  106.38231574,  -37.26902884,   78.1409466 ,
          54.97925142,  183.74418103,  101.34679337,  -15.98975323]])

In [78]:
ypreds = model.predict(X_sel_pre).round(2)
ypreds[:5]

array([[2808.99],
       [3772.  ],
       [3159.35],
       [3462.87],
       [3438.45]])

In [79]:
Y.head()

,Weight
0,2705
1,3560
2,3375
3,3405
4,3640


# Evaluation metrics

In [80]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

mse = mean_squared_error(Y,ypreds)
mae = mean_absolute_error(Y,ypreds)
rmse = mse**(1/2)
r2 = r2_score(Y,ypreds)

print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2 score: {r2*100:.2f}%")

MSE: 13236.86
MAE: 92.19
RMSE: 115.05
R2 score: 96.15%


# Model can be consider for final model building 